In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/12 12:40:12 WARN Utils: Your hostname, hexiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.178.184 instead (on interface en0)
22/09/12 12:40:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 12:40:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
cunsumer_fraud = spark.read.option("header",True).csv('../data/tables/consumer_fraud_probability.csv')
merchant_fraud = spark.read.option("header",True).csv('../data/tables/merchant_fraud_probability.csv')
consumer_transaction = spark.read.option("header",True).parquet('../data/curated/consumer_transaction.parquet')
merchant = spark.read.option("header",True).parquet('../data/curated/merchant.parquet')

In [3]:
cunsumer_fraud.limit(3)

user_id,order_datetime,fraud_probability
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777


In [4]:
from pyspark.sql.functions import *
merchant_fraud = merchant_fraud.select(col("merchant_abn"),col("fraud_probability"),to_date(col("order_datetime"),"yyyy-MM-dd").alias("date"))
cunsumer_fraud = cunsumer_fraud.select(col("user_id"),col("fraud_probability"),to_date(col("order_datetime"),"yyyy-MM-dd").alias("date"))

In [5]:
cunsumer_fraud.limit(3)

user_id,fraud_probability,date
6228,97.6298077657765,2021-12-19
21419,99.24738020302328,2021-12-10
5606,84.05825045251777,2021-10-17


In [6]:
#筛选2021/02/28 - 2022/08/28的数据
print(merchant_fraud.count())
merchant_fraud.filter("date > '2021-02-28'")
merchant_fraud.filter("date < '2022-08-28'")
print(merchant_fraud.count())

print(cunsumer_fraud.count())
cunsumer_fraud.filter("date > '2021-02-28'")
cunsumer_fraud.filter("date < '2022-08-28'")
print(cunsumer_fraud.count())

114
114
34864
34864


In [7]:
consumer_fraud_group = cunsumer_fraud.groupBy('user_id').agg(F.avg('fraud_probability').alias('average_prob_con'), F.count('fraud_probability').alias('fraud_count_con'))
consumer_fraud_group.limit(2)

user_id,average_prob_con,fraud_count_con
7252,37.3984071738018,2
1512,21.912915980669464,3


In [8]:
consumer_transaction = consumer_transaction.join(consumer_fraud_group, consumer_transaction.user_id == consumer_fraud_group.user_id).drop(consumer_fraud_group.user_id)
consumer_transaction = consumer_transaction.withColumn(
    "whether_fraud",
    F.when(F.col('average_prob_con')>=70, 1).otherwise(0))
consumer_transaction.limit(2)

name,address,state,postcode,gender,consumer_id,user_id,avg,sum,count,__index_level_0__,average_prob_con,fraud_count_con
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503,1,155.62835909878035,55092.43912096824,354,0,9.805431136520959,1
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208,2,149.62808995795854,48629.12923633652,325,1,9.834682425100517,2


In [9]:
merchant_fraud_group = merchant_fraud.groupBy('merchant_abn').agg(F.avg('fraud_probability').alias('average_prob_abn'), F.count('fraud_probability').alias('fraud_count_abn'))
merchant_fraud_group.limit(2)

merchant_abn,average_prob_abn,fraud_count_abn
67264251405,18.21089142894488,1
43083074133,48.66890735193894,1


In [10]:
merchant = merchant.join(merchant_fraud_group, merchant.merchant_abn == merchant_fraud_group.merchant_abn).drop(merchant_fraud_group.merchant_abn)
merchant.limit(2)

merchant_abn,Amount,Count,Monthly Amount,Monthly Count,name,tags,Store type,Revenue levels,Take rate,__index_level_0__,average_prob_abn,fraud_count_abn
11149063370,839262.3593570334,46,69938.53,3.83,Et Arcu Limited,((art dealers and...,art dealers and g...,b,4.84,51,53.28693336242285,3
11470993597,16389.683012932896,262,1365.81,21.83,Sed Associates,"((watch, clock, a...","watch, clock, and...",d,1.35,67,63.37734364737917,1
